# More About Type Hints

In [2]:
from typing import TypedDict, TYPE_CHECKING, get_type_hints


class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

In [3]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

In [4]:
def demo() -> None:
    book = BookDict(
        isbn='0134757599', 
        title='Refactoring, 2e', 
        authors=['Martin Fowler', 'Kent Beck'], 
        pagecount=478
    )
    authors = book['authors']
    if TYPE_CHECKING:
        reveal_type(authors)
    authors = 'Bob'
    book['weight'] = 4.2
    del book['title']

In [5]:
demo()

In [6]:
demo.__annotations__

{'return': None}

### to_xml

In [7]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'


def to_xml(book: BookDict) -> str:
    elements: list[str] = []
    for key, value in book.items():
        if isinstance(value, list):
            elements.extend(
                AUTHOR_ELEMENT.format(n) for n in value
            )
        else:
            tag = key.upper()
            elements.append(f'<{tag}>{value}</{tag}>')
    xml = '\n\t'.join(elements)
    return f'<BOOK>\n\t{xml}\n</BOOK>'

In [8]:
to_xml.__annotations__

{'book': __main__.BookDict, 'return': str}

## Generic Class

In [9]:
import random
from collections.abc import Iterable
from typing import TypeVar, Generic

T = TypeVar('T')

class Lotto(Generic[T]):

    def __init__(self, item: Iterable[T]) -> None:
        self._balls = list[T](item)
    
    def load(self, items: Iterable[T]) -> None:
        self._balls.extend(items)
    
    def pick(self) -> T:
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty Lotto')
        return self._balls.pop(position)
    
    def loaded(self) -> bool:
        return bool(self._balls)

    def inspect(self) -> tuple[T, ...]:
        return tuple(self._balls)


### A Covariant Dispenser

Given `A :> B`, a generic type `c` is convariant if `c[A] :> c[B]`

In [11]:
T = TypeVar('T', covariant=True)


class Shape:
    def area(self) -> float:
        pass


class Circle(Shape):
    def area(self) -> float:
        pass


class Square(Shape):
    def area(self) -> float:
        pass


class Container(Generic[T]):
    def __init__(self, item: T):
        self.item = item


circle: Container[Circle] = Container(Circle())
shape: Container[Shape] = circle

### contravariant
Given `A :> B`, a generic type `k` is contravariant if `k[A] :< k[B]`

## A Generic Static Protocol

```python
from typing import Protocol, runtime_checkable, TypeVar


T_co = TypeVar('T_co', covariant=True)


@runtime_checkable
class SupportsAbs(Protocol[T_co]):
    """An ABC with one abstract method __abs__ that is covariant in its
        return type. """
    __slots__ = ()

    @abstractmethod
    def __abs__(self) -> T_co:
        pass
```

In [13]:
import math
from typing import NamedTuple, SupportsAbs


class Vector2d(NamedTuple):
    x: float
    y: float

    def __abs__(self) -> float:
        return math.hypot(self.x, self.y)
    

def is_unit(v: SupportsAbs[float]) -> bool:
    """'True' if the magnitude of 'v' is close to 1. """
    return math.isclose(abs(v), 1.0)


v0 = Vector2d(0, 1)
sqrt2 = math.sqrt(2)
v1 = Vector2d(sqrt2 / 2, sqrt2 / 2)
v2 = Vector2d(1, 1)
v3 = Vector2d(.5, math.sqrt(3) / 2)
v4 = 1

assert is_unit(v0)
assert is_unit(v1)
assert not is_unit(v2)
assert is_unit(v3)
assert is_unit(v4)

print('OK')

OK
